In [19]:
import MySQLdb
import pandas as pd

In [20]:
# Global variables
crf_list = [
        'demographics', 'education', 'hivtestinghistory', 'hivtestreview', 'residencymobility',
        'hivresultdocumentation', 'hivtested', 'hivcareadherence', 'sexualbehaviour', 'recentpartner',
        'secondpartner', 'thirdpartner', 'hivmedicalcare', 'accesstocare', 'circumcision', 'circumcised',
        'uncircumcised', 'reproductivehealth', 'medicaldiagnoses', 'hypertensioncardiovascular', 'heartattack',
        'cancer', 'hivrelatedillness', 'tuberculosis', 'tbsymptoms', 'qualityoflife', 'resourceutilization',
        'outpatientcare', 'hospitaladmission', 'hivhealthcarecosts', 'labourmarketwages',
        'hivlinkagetocare', 'hivresult', 'elisahivresult', 'pimacd4', 'hicenrollment', 'subjectreferral',
        'hivuntested', 'communityengagement', 'pregnancy', 'nonpregnancy', 'stigma', 'stigmaopinion',
        'positiveparticipant', 'substanceuse']

In [21]:
def central_server_frequencies():
    
    # Make a mysql connection
    mysql_cn= MySQLdb.connect(host='127.0.0.1', port=5003,user='root', passwd='cc3721b', db='edc')

    # Plots
    sql_statement = "SELECT * from plot_plot WHERE map_area='mmankgodi';"
    plots_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    plots = plots_df.groupby('map_area').size()
    
    # Visit per survey
    sql_statement = "SELECT * from bcpp_subject_subjectvisit INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_visit_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['visit_code'])    
    visits = subject_visit_df.groupby('visit_code').size()
    
    # Consents
    sql_statement = "SELECT * from bcpp_subject_subjectconsent INNER JOIN member_householdmember ON bcpp_subject_subjectconsent.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_consent_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    consents = subject_consent_df.groupby('map_area').size()
    
    # Household Members
    sql_statement = "SELECT * from member_householdmember INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    household_member_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    members = household_member_df.groupby('map_area').size()
    
    # Requisitions
    sql_statement = "SELECT * FROM bcpp_subject_subjectrequisition INNER JOIN bcpp_subject_subjectvisit ON bcpp_subject_subjectrequisition.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
    subject_requiaitions_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['panel_name', 'visit_code'])
    requisitions = subject_requiaitions_df.groupby(['visit_code']).size()
    
    data_dict = {
        'subject_visits': visits.to_dict(),
        'subject_consents': consents.to_dict(),
        'household_members': members.to_dict(),
        'subject_requisitions': requisitions.to_dict(),
        'plots': plots.to_dict(),
    }
    name_part = 'bcpp_subject_'
    crfs_dict = {}
    for crf in crf_list:
        crf_table_name = name_part + crf
        crf_sql_statement = f"SELECT * FROM {crf_table_name} INNER JOIN bcpp_subject_subjectvisit ON {crf_table_name}.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
        crf_df = pd.read_sql(crf_sql_statement, con=mysql_cn, columns=['visit_code', 'map_area'])
        crf_stats = crf_df.groupby('visit_code').size()
        crfs_dict[crf]=crf_stats.to_dict()
    mysql_cn.close()
    data_dict['crfs'] = crfs_dict
    print('===============================Data preparation complete===============================')
    return data_dict

In [22]:
def community_server_frequencies():
    
    # Make a mysql connection
    mysql_cn= MySQLdb.connect(host='127.0.0.1', port=5002,user='root', passwd='cc3721b', db='mmankgodi')

    # Plots
    sql_statement = "SELECT * from plot_plot WHERE map_area='mmankgodi';"
    plots_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    plots = plots_df.groupby('map_area').size()

    # Visit per survey
    sql_statement = "SELECT * from bcpp_subject_subjectvisit INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_visit_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['visit_code'])    
    visits = subject_visit_df.groupby('visit_code').size()
    
    # Consents
    sql_statement = "SELECT * from bcpp_subject_subjectconsent INNER JOIN member_householdmember ON bcpp_subject_subjectconsent.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    subject_consent_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    consents = subject_consent_df.groupby('map_area').size()
    
    # Household Members
    sql_statement = "SELECT * from member_householdmember INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi';"
    household_member_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['map_area'])    
    members = household_member_df.groupby('map_area').size()
    
    # Requisitions
    sql_statement = "SELECT * FROM bcpp_subject_subjectrequisition INNER JOIN bcpp_subject_subjectvisit ON bcpp_subject_subjectrequisition.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
    subject_requiaitions_df = pd.read_sql(sql_statement, con=mysql_cn, columns=['panel_name', 'visit_code'])
    requisitions = subject_requiaitions_df.groupby(['visit_code']).size()
    
    data_dict = {
        'subject_visits': visits.to_dict(),
        'subject_consents': consents.to_dict(),
        'household_members': members.to_dict(),
        'subject_requisitions': requisitions.to_dict(),
        'plots': plots.to_dict(),
    }

    name_part = 'bcpp_subject_'
    crfs_dict = {}
    for crf in crf_list:
        crf_table_name = name_part + crf
        crf_sql_statement = f"SELECT * FROM {crf_table_name} INNER JOIN bcpp_subject_subjectvisit ON {crf_table_name}.subject_visit_id = bcpp_subject_subjectvisit.id INNER JOIN member_householdmember ON bcpp_subject_subjectvisit.household_member_id = member_householdmember.id INNER JOIN household_householdstructure ON member_householdmember.household_structure_id = household_householdstructure.id INNER JOIN household_household ON household_householdstructure.household_id = household_household.id INNER JOIN plot_plot ON household_household.plot_id = plot_plot.id WHERE plot_plot.map_area='mmankgodi'AND bcpp_subject_subjectvisit.visit_code IN ('T2', 'E0', 'T1', 'T0');"
        crf_df = pd.read_sql(crf_sql_statement, con=mysql_cn, columns=['visit_code', 'map_area'])
        crf_stats = crf_df.groupby('visit_code').size()
        crfs_dict[crf]=crf_stats.to_dict()
    mysql_cn.close()
    data_dict['crfs'] = crfs_dict
    print('===============================Data preparation complete===============================')
    return data_dict

In [23]:
central_server_data = central_server_frequencies()
community_server_data = community_server_frequencies()

===============================Data preparation complete===============================
===============================Data preparation complete===============================


In [24]:
frequency_differences = {}
community = 'mmankgodi'

#household member differences
household_member_diff = community_server_data['household_members'][community] - central_server_data['household_members'][community]
frequency_differences['household_members'] = {
    'central_server': central_server_data['household_members'][community],
    'community_server': community_server_data['household_members'][community],
    'diff': household_member_diff}

#Plots differences
plots_diff = community_server_data['plots'][community] - central_server_data['plots'][community]
frequency_differences['plots'] = {
    'central_server': central_server_data['plots'][community],
    'community_server': community_server_data['plots'][community],
    'diff': plots_diff}

#Subject Consent differences
subject_consents_diff = community_server_data['subject_consents'][community] - central_server_data['subject_consents'][community]
frequency_differences['subject_consents'] = {
    'central_server': central_server_data['subject_consents'][community],
    'community_server': community_server_data['subject_consents'][community],
    'diff': subject_consents_diff}

def dict_diff_by_visit_code(dict_key=None, server_dict=None, community_dict=None):
    visit_codes = ['E0', 'T0', 'T1', 'T2']
    visits_dict = {}
    for visit_code in visit_codes:
        visit_data_dict = {}
        dic1 = community_dict.get(dict_key, '')
        dic2 = server_dict.get(dict_key, '')
        value1 = dic1.get(visit_code, 0)
        value2 = dic2.get(visit_code, 0)
        if value1 and value2:
            diff = value1 - value2
            visit_data_dict[f'community_server_{visit_code}'] = value1
            visit_data_dict[f'central_server_{visit_code}'] = value2
            visit_data_dict[f'community_server_{visit_code}_diff'] = diff
            visits_dict[visit_code] = visit_data_dict
    return visits_dict

# Subject Visit differences
frequency_differences['subject_visits'] = dict_diff_by_visit_code(
    dict_key='subject_visits',
    server_dict=central_server_data,
    community_dict=community_server_data)

# Subject Requisitions

frequency_differences['subject_requisitions'] = dict_diff_by_visit_code(
    dict_key='subject_requisitions',
    server_dict=central_server_data,
    community_dict=community_server_data)

# CRFs
crf_dictionary = {}
def crf_dict_diff_by_visit_code(dict_key=None, server_dict=None, community_dict=None):
    visit_codes = ['E0', 'T0', 'T1', 'T2']
    visits_dict = {}
    for visit_code in visit_codes:
        visit_data_dict = {}
        community_crf = community_dict.get('crfs')
        central_server_crf = server_dict.get('crfs')
        dic1 = community_crf.get(dict_key)
        dic2 = central_server_crf.get(dict_key)
        if dic1 and dic2:
            value1 = dic1.get(visit_code, 0)
            value2 = dic2.get(visit_code, 0)
        else:
            value1 = 0
            value2 = 0
        if value1 and value2:
            diff = value1 - value2
            visit_data_dict[f'community_server_{visit_code}'] = value1
            visit_data_dict[f'central_server_{visit_code}'] = value2
            visit_data_dict[f'community_server_{visit_code}_diff'] = diff
            visits_dict[visit_code] = visit_data_dict
    return visits_dict

for crf in crf_list:
    crf_dictionary[crf] = crf_dict_diff_by_visit_code(
        dict_key=crf, server_dict=central_server_data, community_dict=community_server_data)

In [25]:
from tabulate import tabulate
for key, value in frequency_differences.items():
    df = pd.DataFrame({key: value})
    print(tabulate(df, headers='keys', tablefmt='psql'))

for key, value in crf_dictionary.items():
    df = pd.DataFrame({key: value})
    print(tabulate(df, headers='keys', tablefmt='psql'))

+------------------+---------------------+
|                  |   household_members |
|------------------+---------------------|
| central_server   |                7178 |
| community_server |                7354 |
| diff             |                 176 |
+------------------+---------------------+
+------------------+---------+
|                  |   plots |
|------------------+---------|
| central_server   |    2592 |
| community_server |    2590 |
| diff             |      -2 |
+------------------+---------+
+------------------+--------------------+
|                  |   subject_consents |
|------------------+--------------------|
| central_server   |               3269 |
| community_server |               3446 |
| diff             |                177 |
+------------------+--------------------+
+----+------------------------------------------------------------------------------------------+
|    | subject_visits                                                                     